# Digital markets Act

In [ ]:
# Artikel
# <div id="001">
# Absatz: <div id="001.001"> 

# Que nem o explanation code

In [1]:
import requests 
from bs4 import BeautifulSoup 
import unicodedata
import numpy as np
import pandas as pd

In [2]:
## Getting the law:

URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:32022R1925&from=EN"
r = requests.get(URL) 
soup = BeautifulSoup(r.content, 'html5lib')
law = 'eu_digital_markets_act'

c:\Python310\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [3]:
## Getting all the artikels and separating Artikels with and without Absatze:

#1. getting all artikels and absatze:

ids = [tag['id'] for tag in soup.select('div[id]')]

df = pd.DataFrame(ids)
df.rename(columns={list(df)[0]:'ids'}, inplace=True)
df['artikel'] = df['ids'].str[:3]
df

#2. Looking which one of them have Absatze

df1 = df.groupby(['artikel']).count().reset_index()
df1 = df1.query("ids > 1")
ids_mit = df1['artikel']#.astype('str')

#3. Getting the final ids, so the ids and ids with absatze dont appear twice:
ids_final = set(ids) - set(ids_mit)

## Creating the table with all Artikel and Absatze:

#1. Creating empty df:
df_law = pd.DataFrame(columns = ['original_id','gesetzt', 'artikel', 'absatz', 'text'])

#2. loop for ids:

for id_entry in ids_final:
    #pegando tudo:
    table = soup.find('div', attrs = {'id':id_entry}) 
    my_str = table.text
    result = unicodedata.normalize('NFKD', my_str)   # removind \xa0
    result = " ".join(result.split())
    #result # lindo, está o absatz completo
    #agora pegando os artigos e os absatz
    #criando um df temporario com os detalhes do artigo:
    if len(id_entry) < 4:
        df_temp = pd.DataFrame({'original_id': [id_entry],
                                'gesetzt': [law],  
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : ["NA"],
                                'text': [result]})
    else:
        df_temp = pd.DataFrame({'original_id': [id_entry],
                                'gesetzt': [law],  
                                'artikel': [id_entry.split('.')[0]], 
                                'absatz' : [id_entry.split('.')[1]],
                                'text': [result]})
    #df_temp.dtypes funciona
    df_law = pd.concat([df_law, df_temp])

df_law

,original_id,gesetzt,artikel,absatz,text
0,020.002,eu_digital_markets_act,020,002,(2) Ungeachtet des Absatzes 1 kann die Kommiss...
0,040.004,eu_digital_markets_act,040,004,(4) Die Kommission stellt Sekretariatsdienste ...
0,027.002,eu_digital_markets_act,027,002,(2) Die für die Durchsetzung der in Artikel 1...
0,049.003,eu_digital_markets_act,049,003,(3) Die Befugnisübertragung gemäß Artikel 3 ...
0,041.003,eu_digital_markets_act,041,003,(3) Drei oder mehr Mitgliedstaaten können die...
...,...,...,...,...,...
0,032.004,eu_digital_markets_act,032,004,(4) Nach jeder Unterbrechung beginnt die Frist...
0,013.007,eu_digital_markets_act,013,007,(7) Umgeht der Torwächter eine der Verpflicht...
0,037.001,eu_digital_markets_act,037,001,(1) Die Kommission und die Mitgliedstaaten arb...
0,006.010,eu_digital_markets_act,006,010,(10) Der Torwächter gewährt gewerblichen Nut...


In [4]:
# removing anhang:

df_law = df_law.query('artikel != "L_2022265DE"')

In [5]:
# Salvando

df_law.to_csv("eu_digital_markets_act.csv")